In [6]:
## database imports
from database.market import Market
from database.adatabase import ADatabase

## data processor import
from processor.processor import Processor as p

## risk and return class imports
from returns.products import Products as returns_products_class
from risk.weekly_risk import WeeklyRisk as risk_class
from returns.weekly_returns import WeeklyReturns as returns_class

## backtester import
from backtester.weekly_backtester import WeeklyBacktester as b

## additional imports
from datetime import datetime
import pandas as pd
from tqdm import tqdm

In [2]:
start_date = datetime(2020,1,1)
end_date = datetime(2023,1,1)

In [3]:
market = Market()
speculation_db = ADatabase("btc_spec")
classification_db = ADatabase("btc_spec_classification")
btc_db = ADatabase("btc_weekly")

In [4]:
market.connect()
prices = market.retrieve("crypto")
market.disconnect()
current_prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
current_prices["ticker"] = "BTC"
current_prices = p.column_date_processing(current_prices)
prices = p.column_date_processing(prices)
prices["ticker"] = "BTC"
prices = pd.concat([prices,current_prices])

In [8]:
tyields = returns_products_class.tyields()
bench_returns = returns_products_class.spy_bench()

In [10]:
ticker_sim = prices.copy()
price_returns = returns_class.returns(ticker_sim)
price_returns = risk_class.risk(ticker_sim,bench_returns)

In [11]:
speculation_db.connect()
simulation = speculation_db.retrieve("sim")
speculation_db.disconnect()

In [12]:
classification_db.connect()
classification_simulation = classification_db.retrieve("sim")
classification_db.disconnect()

In [13]:
simulation["week"] = simulation["week"] + 1
classification_simulation["week"] = classification_simulation["week"] + 1
classification_simulation.rename(columns={"prediction":"classification_prediction"},inplace=True)

In [14]:
simulation = price_returns.merge(simulation.drop("adjclose",axis=1)[["year","week","ticker","prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(classification_simulation.drop(["adjclose","training_years"],axis=1)[["year","week","ticker","classification_prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(tyields[["date","weekly_yield"]],on="date",how="left")

In [15]:
sim = simulation.dropna()

In [16]:
sim.sort_values("date",inplace=True)

C:\Users\eric\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [17]:
sim = sim.groupby(["date","ticker","classification_prediction"]).mean().reset_index()

In [18]:
trades = []
values = [True,False]
classifications = [True,False]
ceilings = [True,False]
floors = [True,False]
parameters = []
positions = 1
strats = ["spec","rolling","window"]
for strat in strats:
    for value in values:
        for classification in classifications:
            for ceiling in ceilings:
                for floor in floors:
                    parameter = {
                                "value":value
                                 ,"classification":classification
                                 ,"ceiling":ceiling
                                 ,"floor":floor
                                ,"strat":strat
                                }
                    parameters.append(parameter)

In [19]:
parameter = parameters[0]

In [23]:
btc_db.connect()
btc_db.drop("trades")
for parameter in tqdm(parameters):
    backtest_data = sim.copy().dropna()
    strat = parameter["strat"]
    backtest_data = returns_class.returns_backtest(strat,backtest_data)
    b.backtest(backtest_data.copy(),positions,parameter,start_date,end_date,btc_db)
btc_db.disconnect()

  0%|                                                                                                                                                                   | 0/48 [00:00<?, ?it/s]E:\projects\longshot_venv\longshot\backtester\weekly_backtester.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sim["date_boolean"] = [x.weekday() == 0 for x in sim["date"]]
E:\projects\longshot_venv\longshot\backtester\weekly_backtester.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sim["date_boolean"] = [x.

 25%|██████████████████████████████████████▌                                                                                                                   | 12/48 [00:00<00:01, 22.14it/s]E:\projects\longshot_venv\longshot\backtester\weekly_backtester.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sim["date_boolean"] = [x.weekday() == 0 for x in sim["date"]]
E:\projects\longshot_venv\longshot\backtester\weekly_backtester.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["delta"] = test["del

E:\projects\longshot_venv\longshot\backtester\weekly_backtester.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sim["date_boolean"] = [x.weekday() == 0 for x in sim["date"]]
 50%|█████████████████████████████████████████████████████████████████████████████                                                                             | 24/48 [00:01<00:01, 22.27it/s]E:\projects\longshot_venv\longshot\backtester\weekly_backtester.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sim["date_boolean"] = [x.

 62%|████████████████████████████████████████████████████████████████████████████████████████████████▎                                                         | 30/48 [00:01<00:00, 21.81it/s]E:\projects\longshot_venv\longshot\backtester\weekly_backtester.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sim["date_boolean"] = [x.weekday() == 0 for x in sim["date"]]
E:\projects\longshot_venv\longshot\backtester\weekly_backtester.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["delta"] = test["del

 88%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                   | 42/48 [00:01<00:00, 21.33it/s]E:\projects\longshot_venv\longshot\backtester\weekly_backtester.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sim["date_boolean"] = [x.weekday() == 0 for x in sim["date"]]
E:\projects\longshot_venv\longshot\backtester\weekly_backtester.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["delta"] = test["del